In [1]:
from pprint import pprint
import json, csv
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests, json, csv
from twython import Twython, TwythonStreamer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import twitter_credentials
import tweepy
from pathlib import Path


In [2]:
CONSUMER_KEY = 'Hgr2TlRxnBQwANqQv6lbAPdQK'
CONSUMER_SECRET = 'OjAgBxBOWOUcgAClfuw00KITmy1asnWwVC4DAdOtdDBtOZbtQ2'
ACCESS_TOKEN = '1195204217041903616-Xk76aygVlQ2NjY15dZ5J6SVoGYpHrI'
ACCESS_SECRET= 'r3dkhipY2D3DsqcHGPEUCtsq1CjpbrRzu18odz33pzyxJ'

In [3]:
# # Path directory to save in folder
# path = Path('Users/angy4/BootCamp-HW/Twitter API/Angeles')
# path.mkdir(parents=True)
# directPath = 'userTweets'
# if not os.path.isdir(directPath):
#     os.makedirs(directPath)
    
#     with open('{file_path}.csv'.format(file_path=os.path.join(dir_path, file_name), 'w+') as csv_file:


In [4]:
def get_all_tweets(screen_name):
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)

    
#initialize a list to hold all the tweepy Tweets
    alltweets = []

#make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=3200)

#save most recent tweets
    alltweets.extend(new_tweets)

#save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

#keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        
        print ("getting tweets before %s" % (oldest))

    #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

    #save most recent tweets
        alltweets.extend(new_tweets)

    #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print ("...%s tweets downloaded so far" % (len(alltweets)))

    #transform the tweepy tweets into a 2D array that will populate the csv	
        outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] 
                 for tweet in alltweets]

    #write the csv	
        with open('%s_tweets.json' % screen_name, 'a') as f:
            included_cols = [2]
            json_dumps = json.dumps(outtweets, indent=4, sort_keys=True, default=str)
            f.write(json_dumps)
           
        pass


if __name__ == '__main__':
    #pass in the username of the account you want to download
    
                
    get_all_tweets("BernieSanders")


getting tweets before 1193954435187105793
...400 tweets downloaded so far
getting tweets before 1189232860437368831
...600 tweets downloaded so far
getting tweets before 1185582311103979519
...800 tweets downloaded so far
getting tweets before 1179036995764592639
...1000 tweets downloaded so far
getting tweets before 1174798769428320263
...1200 tweets downloaded so far
getting tweets before 1169964218759688191
...1400 tweets downloaded so far
getting tweets before 1165375074175725567
...1600 tweets downloaded so far
getting tweets before 1162031208387219455
...1800 tweets downloaded so far
getting tweets before 1156696677870383103
...2000 tweets downloaded so far
getting tweets before 1152211199943163908
...2200 tweets downloaded so far
getting tweets before 1148601896921436159
...2400 tweets downloaded so far
getting tweets before 1143588270481260544
...2600 tweets downloaded so far
getting tweets before 1138880296219267071
...2800 tweets downloaded so far
getting tweets before 113497